In [1]:
def ichimoku(df):
    # Conversion line (tenkan)
    period9_high = df['MID_HIGH'].rolling(window=9).max()
    period9_low = df['MID_LOW'].rolling(window=9).min()
    df['conv'] = (period9_high + period9_low) / 2

    # Base line (kijun)
    period26_high = df['MID_HIGH'].rolling(window=26).max()
    period26_low = df['MID_LOW'].rolling(window=26).min()
    df['base'] = (period26_high + period26_low) / 2

    # Leading A (senkou span A, cloud)
    df['cloud1'] = ((df['conv']+df['base'])/2).shift(25)

    # Leading B (senkou span B, cloud)
    period52_high = df['MID_HIGH'].rolling(window=52).max()
    period52_low = df['MID_LOW'].rolling(window=52).min()
    df['cloud2'] = ((period52_high + period52_low) / 2).shift(25)
    # this means the first 76 rows would have NA value under cloud2 column

    # Leading A (senkou span A, future)
    df['lead1'] = (df['conv']+df['base'])/2

    # Leading B (senkou span B, future)
    df['lead2'] = (period52_high + period52_low) / 2

    # Lag (chikou)
    # df['lag'] = df['MID_PRICE'].shift(-24)
    # instead of comparing lagged line with historical price
    # I shift the historical price foward to compare with the currency price
    df['hist_price'] = df['MID_PRICE'].shift(25)

    df = df.dropna()
    df = df.drop(['MID_HIGH','MID_LOW'], axis=1)
    df['signal'] = 'flat'
    
    return df


In [2]:
def add_signal(df):
    for i in df.index:
        if all([df.loc[i]['conv'] > df.loc[i]['base'],
                df.loc[i]['lead1'] > df.loc[i]['lead2'],
                df.loc[i]['MID_PRICE'] > df.loc[i]['cloud1'],
                df.loc[i]['cloud1'] > df.loc[i]['cloud2'],
                df.loc[i]['MID_PRICE'] > df.loc[i]['hist_price']]):
            df.loc[i,['signal']] = 'long'
        else: continue

    # check for short signal
    for i in df.index:
        if all([df.loc[i]['conv'] < df.loc[i]['base'],
                df.loc[i]['lead1'] < df.loc[i]['lead2'],
                df.loc[i]['MID_PRICE'] < df.loc[i]['cloud1'],
                df.loc[i]['cloud1'] < df.loc[i]['cloud2'],
                df.loc[i]['MID_PRICE'] < df.loc[i]['hist_price']]):
            df.loc[i,['signal']] = 'short'
        else: continue
        
    return df

In [3]:
import refinitiv.data as rd
import pandas as pd

rd.open_session()
data = []

security = ['JPY=', 'AUDJPY=', 'NZDJPY=', 'EURJPY=', 'GBPJPY=', 'CADJPY=', 'CHFJPY=']
date_0 = '2023-11-28'
date_1 = '2024-11-28'

for ccy in security:
    markets = rd.get_history(universe=[ccy], fields=['MID_OPEN', 'MID_HIGH', 'MID_LOW','MID_PRICE'],
                             interval="1h", start=date_0, end=date_1
                        )

    df = markets
    df = ichimoku(df)
    df = add_signal(df)
    
    entry = 0
    position = 0
    pnl = []

    for i in df.index:
        if df.loc[i]['signal'] == 'long':
            if position == 0:
                entry = df.loc[i]['MID_OPEN']
                position = 1
                continue
            elif position == -1:
                pnl.append(df.loc[i]['MID_PRICE'] - entry)
                entry = 0
                position = 0
                continue
            else: continue
        elif df.loc[i]['signal'] == 'short':
            if position == 0:
                entry = df.loc[i]['MID_OPEN']
                position = -1
                continue
            elif position == 1:
                pnl.append(entry - df.loc[i]['MID_PRICE'])
                entry = 0
                position = 0
                continue
            else: continue
        elif df.loc[i]['signal'] == 'flat':
            if position == 0:
                continue
            elif position == 1:
                pnl.append(df.loc[i]['MID_PRICE'] - entry)
                entry = 0
                position = 0
                continue
            else: 
                pnl.append(entry - df.loc[i]['MID_PRICE'])
                entry = 0
                position = 0
                continue
    
    data.append([ccy,  round(sum(pnl),2), len(pnl), round(min(pnl),2),round(max(pnl),2),
                round(sum(x>0 for x in pnl)/len(pnl),2), 
                abs(round(sum(x for x in pnl if x>0)/sum(x for x in pnl if x<0),2))])
    
summary = pd.DataFrame(data, columns=['CCY pair', 'total pnl', 
                                      'Number of trades', 
                                      'max drawdown', 'max gain', 'win%','gain/loss'])
display(summary)

,CCY pair,total pnl,Number of trades,max drawdown,max gain,win%,gain/loss
0,JPY=,22.54,190,-2.09,4.54,0.47,1.71
1,AUDJPY=,20.90,205,-0.99,2.80,0.52,2.10
2,NZDJPY=,19.31,176,-0.63,4.10,0.49,2.21
3,EURJPY=,20.53,207,-2.19,3.67,0.51,1.60
4,GBPJPY=,26.00,204,-1.90,4.16,0.46,1.63
5,CADJPY=,-3.81,5,-2.75,1.78,0.20,0.32
6,CHFJPY=,9.92,69,-1.87,3.11,0.57,1.67
